In [1]:
import pandas as pd
import csv
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

In [2]:
model = OllamaLLM(model="llama3")
template = """Question: {question}
Answer: Answer in less than 10 words."""
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | model

In [3]:
chain.invoke("Hello there")

'Nice to meet you, how are you?'

In [3]:
chain.invoke({"question": "Hello, how are you?"})

"I'm good, thanks for asking!"

In [ ]:
template = """
Classify the following item as either a 'fruit' or a 'vegetable':
Item: {item}

Classification:"""

prompt = ChatPromptTemplate.from_template(template)
chain = prompt | model

# Read from CSV and classify
with open('items.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        item = row['item']
        response = chain.invoke({"item": item})
        # Handle the response based on its type
        if isinstance(response, str):
            classification = response.strip()
        else:
            classification = response.content.strip()
        print(f"{item}: {classification}")

In [2]:
#List of fruits and vegetables
items = ["apple", "carrot", "banana", "broccoli", "orange", "tomato", "grape", "cucumber"]

# Create a prompt template for classification
template = """
Classify the following item as either a 'fruit' or a 'vegetable'. Pick 'fruit' or 'vegetable' only. One word :
Item: {item}

Classification:"""

prompt = ChatPromptTemplate.from_template(template)

# Initialize the Ollama model
model = OllamaLLM(model="llama3")

# Create the classification chain
chain = prompt | model

# Iterate through the list and classify each item
for item in items:
    response = chain.invoke({"item": item})

    # Check if response is a string or an object with content attribute
    if isinstance(response, str):
        classification = response.strip()
    else:
        classification = response.content.strip()
    print(f"{item}: {classification}")

In [7]:
# First, let's create a CSV file
items = ["apple", "carrot", "banana", "broccoli", "orange", "tomato", "grape", "cucumber"]

with open('items.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['item'])  # Header
    for item in items:
        writer.writerow([item])

In [9]:
template = """
Classify the following item as either a 'fruit' or a 'vegetable'. Pick 'fruit' or 'vegetable' only. One word :
Item: {item}

Classification:"""

prompt = ChatPromptTemplate.from_template(template)
chain = prompt | model

# Read from CSV, classify, and write results to a new CSV
with open('items.csv', 'r') as infile, open('classified_items.csv', 'w', newline='') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = ['item', 'category']
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        item = row['item']
        response = chain.invoke({"item": item})
        # Handle the response based on its type
        if isinstance(response, str):
            classification = response.strip()
        else:
            classification = response.content.strip()
        
        # Write to the new CSV file
        writer.writerow({'item': item, 'category': classification})
        print(f"Classified {item} as {classification}")

print("Classification complete. Results written to 'classified_items.csv'")

Classified apple as Fruit
Classified carrot as Vegetable
Classified banana as Fruit
Classified broccoli as Vegetable
Classified orange as Fruit
Classified tomato as Fruit
Classified grape as Fruit
Classified cucumber as Vegetable
Classification complete. Results written to 'classified_items.csv'


In [4]:
# URL DATA from data.gov.my
PHARM_URL_DATA = 'https://storage.data.gov.my/healthcare/pharmaceutical_products.parquet'

# Function to read and process dataframe
def read_and_process_df(url):
    df = pd.read_parquet(url)
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])
    return df

# Read and process dataframes
pharm_df = read_and_process_df(PHARM_URL_DATA).iloc[2:]

# Return pharm_df with description column 'PRESCRIPTION','NON PRESCRIPTION' only
rx_pharm = pharm_df[pharm_df['description'].isin(['PRESCRIPTION','NON PRESCRIPTION'])]

In [5]:
rx_pharm

,ref_no,reg_no,product,status,description,holder,holder_osa,manufacturer,manufacturer_osa,importer,importer_osa,date_reg,date_end,active_ingredient,mdc_code,generic_name
2,1997061536X,MAL19991333XZ,ALZENTAL TABLET 400MG,PRODUCT APPROVED,NON PRESCRIPTION,"THE ZYFAS MEDICAL CO.,",929,SHIN POONG PHARM CO LTD,73798,THE ZYFAS PHARMA SDN. BHD.,1267052.0,1559260800000000000,1722902400000000000,ALBENDAZOLE[400MG ],None,None
5,201109122012A,MAL14075063AZ,BETANOR 16MG TABLET,PRODUCT APPROVED,PRESCRIPTION,Noripharma Sdn. Bhd.,46304,Noripharma Sdn. Bhd.,1020114,None,NaN,1564012800000000000,1721865600000000000,BETAHISTINE DIHYDROCHLORIDE[16mg;0],N07CA01-110-T34-02-002,None
39,2004052657351,MAL06111293XZ,Tonbien Solution “Jen Sheng”,PRODUCT APPROVED,NON PRESCRIPTION,PEMBEKALAN UBAT HUATHAI SDN. BHD.,839,JEN SHENG PHARM CO LTD,74065,PEMBEKALAN UBAT HUATHAI SDN. BHD.,55850.0,1564617600000000000,1722470400000000000,Sodium Chloride[0.15;g;1;ml],None,None
45,2001086418A,MAL20040567AZ,KETO PLUS SHAMPOO,CONDITIONAL REGISTRATION,PRESCRIPTION,Glenmark Pharmaceuticals (Malaysia) Sdn. Bhd.,4619,GLENMARK PHARMACEUTICALS LTD,1017943,Glenmark Pharmaceuticals (Malaysia) Sdn. Bhd.,1133209.0,1564617600000000000,1722470400000000000,ZINC PYRITHIONE 48%[1% ...,None,None
51,201802020050A,MAL19086001ACZ,"Corneax, eye drops solution 0.2% w/v",PRODUCT APPROVED,PRESCRIPTION,AVERROES PHARMACEUTICALS SDN. BHD.,804,COOPER S.A.,1116163,AVERROES PHARMACEUTICALS SDN. BHD.,55994.0,1564617600000000000,1722470400000000000,"BRIMONIDINE TARTRATE[2 mg,1 ml]",S01EA05-123-D20-02-004,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27143,1988110068X,MAL19950130XZ,MUCOXIN TABLET 8MG,PRODUCT APPROVED,NON PRESCRIPTION,DUOPHARMA (M) SDN. BHD.,55800,DUOPHARMA (M) SDN. BHD.,1020100,None,NaN,1732924800000000000,1890691200000000000,Bromhexine HCl[8MG ],None,None
27145,1988110079X,MAL19950137XZ,TEMPOL TABLETS 500MG,PRODUCT APPROVED,NON PRESCRIPTION,DUOPHARMA (M) SDN. BHD.,55800,DUOPHARMA (M) SDN. BHD.,1020100,None,NaN,1732924800000000000,1890691200000000000,Paracetamol[500MG ],None,None
27156,201907052343X,MAL19126029XCZ,K-Worm Choc Chewtab 200 mg,PRODUCT APPROVED,NON PRESCRIPTION,Powerlife (M) Sdn. Bhd.,50835,WINWA MEDICAL SDN. BHD.,945200,None,NaN,1733443200000000000,1891209600000000000,ALBENDAZOLE[200 mg],P02CA03-000-T20-02-002,None
27157,201303182014X,MAL14125077X,UNI-ISABGOL (Ispaghula Husk Oral Powder),PRODUCT APPROVED,NON PRESCRIPTION,KCK PHARMACEUTICAL INDUSTRIES SDN. BHD.,779,KCK PHARMACEUTICAL INDUSTRIES SDN. BHD.,1019503,None,NaN,1733443200000000000,1891209600000000000,PSYLLIUM HUSK[100% w/w],None,None


In [10]:
test_data = rx_pharm[['reg_no','product','description','active_ingredient', 'generic_name']]
test_data = test_data.iloc[0:10]
test_data

,reg_no,product,description,active_ingredient,generic_name
2,MAL19991333XZ,ALZENTAL TABLET 400MG,NON PRESCRIPTION,ALBENDAZOLE[400MG ],None
5,MAL14075063AZ,BETANOR 16MG TABLET,PRESCRIPTION,BETAHISTINE DIHYDROCHLORIDE[16mg;0],None
39,MAL06111293XZ,Tonbien Solution “Jen Sheng”,NON PRESCRIPTION,Sodium Chloride[0.15;g;1;ml],None
45,MAL20040567AZ,KETO PLUS SHAMPOO,PRESCRIPTION,ZINC PYRITHIONE 48%[1% ...,None
51,MAL19086001ACZ,"Corneax, eye drops solution 0.2% w/v",PRESCRIPTION,"BRIMONIDINE TARTRATE[2 mg,1 ml]",None
56,MAL09072750AZ,QUACE ECZEMA OINTMENT,PRESCRIPTION,CLOBETASOL PROPIONATE[0.5;mg;1.0;g],Clobetasol Propionate 0.05 % Ointment
66,MAL09072753AZ,Kloraxin Eye Ointment 1% w/w,PRESCRIPTION,Chloramphenicol B.P.[10.00;mg;;g],Chloramphenicol 1% Eye Ointment
73,MAL19991713AZ,Vinblastine Sulfate Kabi Injection 1mg/ml,PRESCRIPTION,VINBLASTINE SULPHATE[1MG/MLX10ML ...,None
77,MAL09072786ACZ,TAZPEN for Injection 4.5g,PRESCRIPTION,"PIPERACILLIN SODIUM[4.184;g;;0],TAZOBACTAM SOD...",Piperacillin 4 g & Tazobactam 500 mg Injection
80,MAL09072790AZ,Resicalm Solution 1 mg/ml,PRESCRIPTION,RISPERIDONE[1;mg;1;ml],Risperidone 1 mg/ml Oral Solution


In [8]:
template = """
Classify the following medication based on general classification. Pick a group of the classification without explaining anything. One to two words only.:
Medication: {product}
Active Ingredient: {active_ingredient}

Classification:"""

prompt = ChatPromptTemplate.from_template(template)
chain = prompt | model

# Function to classify a single medication and print the result
def classify_and_print(row):
    response = chain.invoke({
        "product": row['product'],
        "active_ingredient": row['active_ingredient']
    })
    classification = response.strip() if isinstance(response, str) else response.content.strip()
    print(f"Medication: {row['product']}")
    print(f"Active Ingredient: {row['active_ingredient']}")
    print(f"Classification: {classification}")
    print("-" * 50)
    return classification

# Iterate through each row and classify
classifications = []
for index, row in test_data.iterrows():
    classification = classify_and_print(row)
    classifications.append(classification)

# Add classifications to the DataFrame
test_data['classification'] = classifications

# Display the results
print(test_data[['product', 'active_ingredient', 'classification']])

Medication: ALZENTAL TABLET 400MG
Active Ingredient: ALBENDAZOLE[400MG                         ]
Classification: Anti-Helminthic
--------------------------------------------------
Medication: BETANOR 16MG TABLET
Active Ingredient: BETAHISTINE DIHYDROCHLORIDEg;0]
Classification: Histamine antagonist
--------------------------------------------------
Medication: Tonbien Solution “Jen Sheng”
Active Ingredient: Sodium Chloride[0.15;g;1;ml]
Classification: Electrolyte
--------------------------------------------------
Medication: KETO PLUS SHAMPOO
Active Ingredient: ZINC PYRITHIONE 48%[1%                            ],KETOCONAZOLE[2%                            ]
Classification: Topical Antifungal
--------------------------------------------------
Medication: Corneax, eye drops solution 0.2% w/v
Active Ingredient: BRIMONIDINE TARTRATE[2 mg,1 ml]
Classification: CNS Medication
--------------------------------------------------
Medication: QUACE ECZEMA OINTMENT
Active Ingredient: CLOBETASOL PRO

KeyboardInterrupt: 